# POC ETL_1 - Prototype découpé en fonction

Vous trouverez les codes sources des fonction dans à ces chemins : 
- [extract.py](extract.py)
- [transform.py](transform.py) 

## CONFIGS

In [1]:
import os
if ".env" not in os.listdir():
    os.chdir("data4good/13_brigade_coupes_rases/data_pipeline")

In [2]:
from transform import *
from extract import *

In [3]:
# Vérification du .env
load_dotenv(f".env")
env_vars = ["AWS_SECRET_ACCESS_KEY", "AWS_ACCESS_KEY_ID"]

for var in env_vars:
    try:
        value = os.environ[var]  
        print(f"✅ {var} est bien chargé.")
    except KeyError:
        print(f"⚠️ Attention : {var} n'est pas défini dans l'environnement !")

✅ AWS_SECRET_ACCESS_KEY est bien chargé.
✅ AWS_ACCESS_KEY_ID est bien chargé.


In [4]:
# Variables 
bucket_name = "brc-poc-etl"
api_query = "mosaics_tropisco_warnings_france_date.tif"
download_path = "temp_tif/mosaics_tropisco_warnings_france_date.tif"
s3_key = "raw_data/raster/sufosat_data/mosaics_tropisco_warnings_france_date.tif"
s3_key_metadata = "raw_data/raster/sufosat_data/mosaics_tropisco_warnings_france_date_metadata.json"
url = "https://zenodo.org/records/13685177/files/mosaics_tropisco_warnings_france_date.tif?download=1"

s3 = boto3.client(
    service_name = "s3",
    region_name = "eu-west-3",
    aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY"),
)

## PARTIE EXTRACT DE L'ETL

In [5]:
data, do_update = data_update(api_query, bucket_name, s3_key=s3_key, s3_key_metadata=s3_key_metadata)

✅ Inspection de la metadata effectuée, mise à jour requise : False


In [6]:
if do_update or check_tif_in_s3(bucket_name, s3_key):
    print("✅ Les données sont déjà à jour, pas besoin de télécharger à nouveau")
else:
    extract_tif_data_and_upload(url, s3_key, s3)

✅ Les données sont déjà à jour, pas besoin de télécharger à nouveau


## PARTIE TRANSFORM DE L'ETL

In [7]:
if check_tif_in_s3(bucket_name, s3_key):
    load_from_S3(bucket_name, s3_key, download_path)
else:
    print("❌ Le fichier n'est pas sur S3")

✅ Fichier téléchargé depuis S3 temp_tif/mosaics_tropisco_warnings_france_date.tif


In [8]:
regroup_sufosat_days(
    # Download from "mosaics_tropisco_warnings_france_date.tif" from https://zenodo.org/records/13685177
    "temp_tif/mosaics_tropisco_warnings_france_date.tif",
    "temp_tif/mosaics_tropisco_warnings_france_date_2024.tif",
    pd.Timestamp(year=2024, month=1, day=1),
    pd.Timestamp(year=2024, month=12, day=31),
)

15458it [00:53, 289.56it/s]                                                                                                                    

✅ Fichier tif regroupé


In [9]:
extract_raster_as_geodataframe(
    raster_path = "temp_tif/mosaics_tropisco_warnings_france_date_2024.tif",
    vector_path = "temp_shape/mosaics_tropisco_warnings_france_date_2024.shp"
)

✅ Raster ouvert avec Gdal
✅ Fichier shapefile créé
✅ Couche SRS créée
✅ Polygonisation du raster réussie


In [10]:
sufosat_2024: gpd.GeoDataFrame = gpd.read_file(f"temp_shape")
sufosat_2024: gpd.GeoDataFrame = transform_raster(sufosat_2024)
geojson_bytes = convert_geometries_to_wkt(sufosat_2024)

✅ Calcul de la surface en hectares
✅ Calcul des coordonnées
✅ Conversion des géométries en WKT
✅ Conversion bytes GEOJSON


## PARTIE LOAD DE L'ETL --> PAS COMPLET 
> Les données sont chargées sur S3 pas encore dans la base. Un autre pipe pour le vrai load dans la database

In [11]:
upload_geodataframe_to_s3(
    geojson_bytes, 
    bucket_name, 
    s3
)

✅ Fichier GeoJSON upload sur S3


### Afficher les données uploaded sur S3

In [12]:
s3 = boto3.client('s3')
result = s3.list_objects_v2(Bucket=bucket_name)

if "Contents" in result:
    print("Voici les objets présents dans le bucket :")
    for obj in result["Contents"]:
        print(f"- {obj['Key']}")

Voici les objets présents dans le bucket :
- raw_data/raster/sufosat_data/mosaics_tropisco_warnings_france_date.tif
- raw_data/raster/sufosat_data/mosaics_tropisco_warnings_france_date_metadata.json
- to_api/clear_cut_processed_2025-02-19.geojson
